In [7]:
import json
import os
import pandas as pd
import numpy as np

In [35]:
from itertools import product

def generate_triggers(condition_dict):
    """
    Generate a dictionary of trigger definitions based on combinations of condition levels.

    Parameters:
        condition_dict (dict): A dictionary where keys are factor names (e.g., 'v_stimulus')
                               and values are lists of condition levels for that factor.

    Returns:
        triggers (dict): A dictionary with trial_type strings as keys, and sub-dictionaries
                         containing trigger_id and event codes.
        num_triggers (int): Total number of unique trial combinations.
    """
    # Extract all factor names and corresponding level lists
    factor_names = list(condition_dict.keys())
    level_lists = list(condition_dict.values())

    # Compute all combinations (Cartesian product)
    combinations = list(product(*level_lists))
    num_triggers = len(combinations)
    
    triggers = {}
    for i, levels in enumerate(combinations, start=1):
        # Create trial_type string
        trial_type = "_".join(str(level) for level in levels)
        
        # Define sub-dictionary with trigger structure
        triggers[trial_type] = {
            "trial_start": i,
            "cue_onset": i + num_triggers,
            "isi": i + 2 * num_triggers,
            "target_onset": i + 3 * num_triggers,
            "response_onset": i + 4 * num_triggers,
        }

    return triggers


In [36]:
# Define conditions
conditions = {
    "v_stimulus": [45, 135],
    "v_pred_cond": ["EXP", "UEX", "neutral"],
    "a_stimulus": [100, 160],
    "a_pred_cond": ["EXP", "UEX"],
}
# Generate triggers
triggers = generate_triggers(conditions)
# Convert nested dictionary to DataFrame with trial_type as the index
triggers_df = pd.DataFrame.from_dict(triggers, orient='index')
triggers_df.to_csv("triggers.csv", index=True, header=True)

In [37]:
def generate_triggers(condition_dict):
    """
    Generate a dictionary of trigger definitions based on combinations of condition levels.

    Parameters:
        condition_dict (dict): A dictionary where keys are factor names (e.g., 'v_stimulus')
                               and values are lists of condition levels for that factor.

    Returns:
        triggers (dict): A dictionary with trial_type + event as strings as keys and trigger numbers as values.
    """
    # Extract all factor names and corresponding level lists
    factor_names = list(condition_dict.keys())
    level_lists = list(condition_dict.values())

    # Compute all combinations (Cartesian product)
    combinations = list(product(*level_lists))
    num_triggers = len(combinations)
    
    triggers = {}
    for i, levels in enumerate(combinations, start=1):
        # Create trial_type string
        trial_type = "_".join(str(level) for level in levels)
        
        # Define sub-dictionary with trigger structure
        triggers[f"{trial_type}_trial_start"] = i 
        triggers[f"{trial_type}_cue_onset"] = i + num_triggers
        triggers[f"{trial_type}_isi"] = i + 2 * num_triggers
        triggers[f"{trial_type}_target_onset"] = i + 3 * num_triggers
        triggers[f"{trial_type}_response"] = i + 4 * num_triggers

    return triggers

In [38]:
# Define conditions
conditions = {
    "v_stimulus": [45, 135],
    "v_pred_cond": ["EXP", "UEX", "neutral"],
    "a_stimulus": [100, 160],
    "a_pred_cond": ["EXP", "UEX"],
}
# Generate triggers
triggers = generate_triggers(conditions)

In [39]:
triggers

{'45_EXP_100_EXP_trial_start': 1,
 '45_EXP_100_EXP_cue_onset': 25,
 '45_EXP_100_EXP_isi': 49,
 '45_EXP_100_EXP_target_onset': 73,
 '45_EXP_100_EXP_response': 97,
 '45_EXP_100_UEX_trial_start': 2,
 '45_EXP_100_UEX_cue_onset': 26,
 '45_EXP_100_UEX_isi': 50,
 '45_EXP_100_UEX_target_onset': 74,
 '45_EXP_100_UEX_response': 98,
 '45_EXP_160_EXP_trial_start': 3,
 '45_EXP_160_EXP_cue_onset': 27,
 '45_EXP_160_EXP_isi': 51,
 '45_EXP_160_EXP_target_onset': 75,
 '45_EXP_160_EXP_response': 99,
 '45_EXP_160_UEX_trial_start': 4,
 '45_EXP_160_UEX_cue_onset': 28,
 '45_EXP_160_UEX_isi': 52,
 '45_EXP_160_UEX_target_onset': 76,
 '45_EXP_160_UEX_response': 100,
 '45_UEX_100_EXP_trial_start': 5,
 '45_UEX_100_EXP_cue_onset': 29,
 '45_UEX_100_EXP_isi': 53,
 '45_UEX_100_EXP_target_onset': 77,
 '45_UEX_100_EXP_response': 101,
 '45_UEX_100_UEX_trial_start': 6,
 '45_UEX_100_UEX_cue_onset': 30,
 '45_UEX_100_UEX_isi': 54,
 '45_UEX_100_UEX_target_onset': 78,
 '45_UEX_100_UEX_response': 102,
 '45_UEX_160_EXP_trial_st